In [1]:
!echo > requirements.txt


In [4]:
# importando as bibliotecas necessarias
import json
from datetime import datetime
from collections import defaultdict
from typing import List, Tuple
import ijson
from collections import Counter
import emoji
import re


In [5]:
def format_json(file_path: str) -> list:
    """Lê um arquivo que contém múltiplos objetos JSON (por exemplo, JSON lines) e retorna uma lista de objetos JSON válidos."""

    formatted_data = []

    with open(file_path, 'r') as f:
        for line in f:
            try:
                # Tenta carregar a linha como um objeto JSON
                json_obj = json.loads(line.strip())
                formatted_data.append(json_obj)
            except json.JSONDecodeError:
                # Se a linha não for um JSON válido, simplesmente a ignore
                continue

    return formatted_data

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    """Retorna as 10 principais datas com mais tweets e o usuário com mais tweets em cada uma dessas datas."""

    # Chame format_json para obter os dados formatados do arquivo
    data = format_json(file_path)

    # Inicializar um defaultdict para contabilizar os tweets por data e usuário
    counts = defaultdict(lambda: defaultdict(int))

    for tweet in data:
        # Extrair a data (sem a hora) e o nome do usuário do tweet
        date_str = tweet['date'].split('T')[0]
        user = tweet['user']['username']
        date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()

        # Atualizar o contador para a combinação de data e usuário
        counts[date_obj][user] += 1

    # Obter as 10 datas com o maior número total de tweets
    sorted_dates = sorted(counts.keys(), key=lambda x: sum(counts[x].values()), reverse=True)[:10]

    # Para cada uma dessas datas, obter o usuário com mais tweets
    result = [(date, max(counts[date].keys(), key=lambda x: counts[date][x])) for date in sorted_dates]

    return result


In [6]:
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    """Retorna as 10 principais datas com mais tweets e o usuário com mais tweets em cada uma dessas datas, otimizado para o uso de memória."""

    # Inicializar um defaultdict para contabilizar os tweets por data e usuário
    counts = defaultdict(lambda: defaultdict(int))

    with open(file_path, 'r') as f:
        for line in f:
            try:
                # Tenta carregar a linha como um objeto JSON
                tweet = json.loads(line.strip())

                # Extrair a data (sem a hora) e o nome do usuário do tweet
                date_str = tweet['date'].split('T')[0]
                user = tweet['user']['username']
                date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()

                # Atualizar o contador para a combinação de data e usuário
                counts[date_obj][user] += 1

            except json.JSONDecodeError:
                # Se a linha não for um JSON válido, simplesmente a ignore
                continue

    # Obter as 10 datas com o maior número total de tweets
    sorted_dates = sorted(counts.keys(), key=lambda x: sum(counts[x].values()), reverse=True)[:10]

    # Para cada uma dessas datas, obter o usuário com mais tweets
    result = [(date, max(counts[date].keys(), key=lambda x: counts[date][x])) for date in sorted_dates]

    return result


In [7]:
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    """Retorna os 10 emojis mais usados nos tweets, decodificando emojis codificados em JSON."""

    # Função interna para formatar os dados JSON
    def format_json(file_path: str) -> list:
        """Lê um arquivo que contém múltiplos objetos JSON e retorna uma lista de objetos JSON válidos."""
        formatted_data = []

        with open(file_path, 'r') as f:
            for line in f:
                try:
                    # Tenta carregar a linha como um objeto JSON
                    json_obj = json.loads(line.strip())
                    formatted_data.append(json_obj)
                except json.JSONDecodeError:
                    # Se a linha não for um JSON válido, simplesmente a ignore
                    continue

        return formatted_data

    # Inicializar um contador para os emojis
    emoji_counter = Counter()

    # Chame format_json para obter os dados formatados do arquivo
    data = format_json(file_path)

    for tweet in data:
        # Supondo que o campo 'content' contenha o conteúdo do tweet
        tweet_text = tweet.get('content', '')

        # Decodificar os escapes Unicode para obter o texto real
        decoded_text = tweet_text.encode('utf-16', 'surrogatepass').decode('utf-16')

        # Extrair todos os emojis do tweet
        emojis_list = [char for char in decoded_text if char in emoji.UNICODE_EMOJI_ENGLISH]

        # Atualizar o contador de emojis
        emoji_counter.update(emojis_list)

    # Obter os 10 emojis mais frequentes
    top_10_emojis = emoji_counter.most_common(10)

    return top_10_emojis


In [8]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    """Retorna os 10 emojis mais usados nos tweets, otimizado para o uso de memória, decodificando emojis codificados em JSON."""

    # Inicializar um contador para os emojis
    emoji_counter = Counter()

    with open(file_path, 'r', encoding="utf-8") as f:
        for line in f:
            try:
                # Tenta carregar a linha como um objeto JSON
                tweet = json.loads(line.strip())

                # Decodificamos os escapes Unicode para obter o texto real do tweet
                tweet_text = tweet['content'].encode('utf-16', 'surrogatepass').decode('utf-16')

                # Extrair todos os emojis do tweet
                emojis_list = [char for char in tweet_text if char in emoji.UNICODE_EMOJI_ENGLISH]

                # Atualizar o contador de emojis
                emoji_counter.update(emojis_list)

            except json.JSONDecodeError:
                # Se a linha não for um JSON válido, simplesmente a ignore
                continue

    # Obter os 10 emojis mais frequentes
    top_10_emojis = emoji_counter.most_common(10)

    return top_10_emojis


In [9]:
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    """Retorna os 10 usuários mais mencionados nos tweets."""

    def format_json(file_path: str) -> list:
        """Lê um arquivo que contém múltiplos objetos JSON e retorna uma lista de objetos JSON válidos."""
        formatted_data = []

        with open(file_path, 'r') as f:
            for line in f:
                try:
                    json_obj = json.loads(line.strip())
                    formatted_data.append(json_obj)
                except json.JSONDecodeError:
                    continue

        return formatted_data

    mention_pattern = re.compile(r"@(\w+)")  # Padrão para identificar menções
    mention_counter = Counter()  # Contador para menções

    data = format_json(file_path)

    for tweet in data:
        tweet_text = tweet.get('content', '')
        mentions = mention_pattern.findall(tweet_text)
        mention_counter.update(mentions)

    top_10_mentions = mention_counter.most_common(10)

    return top_10_mentions


In [10]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    """Retorna os 10 usuários mais mencionados nos tweets, otimizado para uso da memória."""

    mention_pattern = re.compile(r"@(\w+)")  # Padrão para identificar menções
    mention_counter = Counter()  # Contador para menções

    with open(file_path, 'r', encoding="utf-8") as f:
        for line in f:
            try:
                tweet = json.loads(line.strip())
                tweet_text = tweet.get('content', '')

                mentions = mention_pattern.findall(tweet_text)
                mention_counter.update(mentions)

            except json.JSONDecodeError:
                continue

    top_10_mentions = mention_counter.most_common(10)

    return top_10_mentions


In [11]:

def main():
    # Pede ao usuário o caminho do arquivo JSON
    file_path = input("Digite o caminho do arquivo JSON: ")

    # Chama a função q1_time e imprime o resultado
    print("\n10 principais datas com mais tweets e o usuário com mais publicações para cada uma (baseado em tempo):")
    for date, user in q1_time(file_path):
        print(date, user)

    # # Chama a função q1_memory e imprime o resultado
    print("\n10 principais datas com mais tweets e o usuário com mais publicações para cada uma (otimizado para memória):")
    for date, user in q1_memory(file_path):
        print(date, user)

    # Chama a função q2_time e imprime o resultado
    print("\n10 emojis mais usados ​​(baseado em tempo):")
    for emoji_char, count in q2_time(file_path):
        print(emoji_char, count)

    # # Chama a função q2_memory e imprime o resultado
    print("\n10 emojis mais usados (otimizado para memória):")
    for emoji_char, count in q2_memory(file_path):
        print(emoji_char, count)

    # # Chama a função q3_time e imprime o resultado
    print("\n10 usuários mais mencionados (baseado em tempo):")
    for user, count in q3_time(file_path):
        print(user, count)

    # # Chama a função q3_memory e imprime o resultado
    print("\n10 usuários mais mencionados (otimizado para memória):")
    for user, count in q3_memory(file_path):
        print(user, count)

# Executa a função main quando o script é executado
if __name__ == "__main__":
    main()


Digite o caminho do arquivo JSON: /content/sample_data/farmers-protest-tweets-2021-2-4.json

10 principais datas com mais tweets e o usuário com mais publicações para cada uma (baseado em tempo):
2021-02-12 RanbirS00614606
2021-02-13 MaanDee08215437
2021-02-17 RaaJVinderkaur
2021-02-16 jot__b
2021-02-14 rebelpacifist
2021-02-18 neetuanjle_nitu
2021-02-15 jot__b
2021-02-20 MangalJ23056160
2021-02-23 Surrypuria
2021-02-19 Preetm91

10 principais datas com mais tweets e o usuário com mais publicações para cada uma (otimizado para memória):
2021-02-12 RanbirS00614606
2021-02-13 MaanDee08215437
2021-02-17 RaaJVinderkaur
2021-02-16 jot__b
2021-02-14 rebelpacifist
2021-02-18 neetuanjle_nitu
2021-02-15 jot__b
2021-02-20 MangalJ23056160
2021-02-23 Surrypuria
2021-02-19 Preetm91

10 emojis mais usados ​​(baseado em tempo):


<ipython-input-7-48ac6ea4e0e8>:35: DeprecationWarning: 'emoji.UNICODE_EMOJI_ENGLISH' is deprecated and will be removed in version 2.0.0. Use 'emoji.EMOJI_DATA' instead.
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  emojis_list = [char for char in decoded_text if char in emoji.UNICODE_EMOJI_ENGLISH]


🙏 7286
😂 3072
🚜 2972
✊ 2411
🌾 2363
🏻 2080
❤ 1779
🤣 1668
🏽 1218
👇 1108

10 emojis mais usados (otimizado para memória):


<ipython-input-8-d73ce77b3fd8>:17: DeprecationWarning: 'emoji.UNICODE_EMOJI_ENGLISH' is deprecated and will be removed in version 2.0.0. Use 'emoji.EMOJI_DATA' instead.
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  emojis_list = [char for char in tweet_text if char in emoji.UNICODE_EMOJI_ENGLISH]


🙏 7286
😂 3072
🚜 2972
✊ 2411
🌾 2363
🏻 2080
❤ 1779
🤣 1668
🏽 1218
👇 1108

10 usuários mais mencionados (baseado em tempo):
narendramodi 2261
Kisanektamorcha 1836
RakeshTikaitBKU 1639
PMOIndia 1422
RahulGandhi 1125
GretaThunberg 1046
RaviSinghKA 1015
rihanna 972
UNHumanRights 962
meenaharris 925

10 usuários mais mencionados (otimizado para memória):
narendramodi 2261
Kisanektamorcha 1836
RakeshTikaitBKU 1639
PMOIndia 1422
RahulGandhi 1125
GretaThunberg 1046
RaviSinghKA 1015
rihanna 972
UNHumanRights 962
meenaharris 925
